In [1]:
import time
import json
import logging
import requests
import pandas as pd
from pprint import pprint
from bs4 import BeautifulSoup

In [2]:
LOG_FILENAME = "lastlog.log"
JSON_FILENAME = "ncov_{ctime}.json".format(ctime=time.strftime('%Y%m%d-%H%M%S'))
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
with open(LOG_FILENAME, 'w') as f:
    f.writelines([LOG_FORMAT, '\n'])
logging.basicConfig(filename=LOG_FILENAME, level=logging.DEBUG, format=LOG_FORMAT)

logging.debug("This is a debug log.")
logging.info("This is a info log.")
logging.warning("This is a warning log.")
logging.error("This is a error log.")
logging.critical("This is a critical log.")

In [3]:
response = requests.get("https://ncov.dxy.cn/ncovh5/view/pneumonia")
HTML = response.text.encode(response.encoding).decode()
soup = BeautifulSoup(HTML)
soup.prettify

9000,"tags":"确诊 2 例","countryType":1,"continents":"","provinceId":"67","provinceName":"澳门","provinceShortName":"澳门","cityName":"","confirmedCount":10,"suspectedCount":0,"curedCount":1,"deadCount":0,"comment":"","sort":23,"operator":"zyyun","locationId":820000},{"id":24,"createTime":1579617167000,"modifyTime":1581239335000,"tags":"确诊 1 例","countryType":1,"continents":"","provinceId":"68","provinceName":"台湾","provinceShortName":"台湾","cityName":"","confirmedCount":18,"suspectedCount":0,"curedCount":1,"deadCount":0,"comment":"","sort":27,"operator":"huanshi","locationId":710000},{"id":182,"createTime":1579860584000,"modifyTime":1580955107000,"tags":"","countryType":1,"continents":"","provinceId":"63","provinceName":"青海省","provinceShortName":"青海","cityName":"","confirmedCount":18,"suspectedCount":0,"curedCount":3,"deadCount":0,"comment":"","sort":40,"operator":"xuyt","locationId":630000},{"id":25,"createTime":1579617215000,"modifyTime":1581292642000,"tags":"确诊 2 例 疑似 65 例","countryType":1,"

In [4]:
text = soup.contents[1].text
def parse_text(text):
    subtext = text[text.find("window.getAreaStat")::]
    subtext = subtext[subtext.find("[{")::]
    # print(subtext)
    num = 0
    chars = []
    for c in subtext:
        chars.append(c)
        if c == '[':
            num += 1
        if c == ']':
            num -= 1
        if num == 0:
            break
    return ''.join(chars)
counting_json = json.loads(parse_text(text))
print(counting_json)

ount': 0, 'locationId': 0}]}, {'provinceName': '广西壮族自治区', 'provinceShortName': '广西', 'confirmedCount': 210, 'suspectedCount': 0, 'curedCount': 18, 'deadCount': 1, 'comment': '', 'locationId': 450000, 'cities': [{'cityName': '南宁', 'confirmedCount': 37, 'suspectedCount': 0, 'curedCount': 2, 'deadCount': 0, 'locationId': 450100}, {'cityName': '北海', 'confirmedCount': 35, 'suspectedCount': 0, 'curedCount': 3, 'deadCount': 0, 'locationId': 450500}, {'cityName': '桂林', 'confirmedCount': 30, 'suspectedCount': 0, 'curedCount': 5, 'deadCount': 0, 'locationId': 450300}, {'cityName': '柳州', 'confirmedCount': 22, 'suspectedCount': 0, 'curedCount': 3, 'deadCount': 0, 'locationId': 450200}, {'cityName': '河池', 'confirmedCount': 21, 'suspectedCount': 0, 'curedCount': 0, 'deadCount': 1, 'locationId': 451200}, {'cityName': '防城港', 'confirmedCount': 17, 'suspectedCount': 0, 'curedCount': 1, 'deadCount': 0, 'locationId': 450600}, {'cityName': '来宾', 'confirmedCount': 11, 'suspectedCount': 0, 'curedCount': 0, '

In [5]:
country_df = pd.read_json(json.dumps(counting_json))
country_df.to_json(open(JSON_FILENAME, 'w'))
country_df

,provinceName,provinceShortName,confirmedCount,suspectedCount,curedCount,deadCount,comment,locationId,cities
0,湖北省,湖北,29631,0,1795,871,,420000,"[{'cityName': '武汉', 'confirmedCount': 16902, '..."
1,广东省,广东,1151,0,143,1,,440000,"[{'cityName': '深圳', 'confirmedCount': 368, 'su..."
2,浙江省,浙江,1092,0,201,0,,330000,"[{'cityName': '温州', 'confirmedCount': 464, 'su..."
3,河南省,河南,1073,0,169,6,参考卫健委统计口径，部分县区与地级市合并,410000,"[{'cityName': '信阳', 'confirmedCount': 220, 'su..."
4,湖南省,湖南,879,0,186,1,,430000,"[{'cityName': '长沙', 'confirmedCount': 213, 'su..."
5,安徽省,安徽,830,0,73,3,,340000,"[{'cityName': '合肥', 'confirmedCount': 147, 'su..."
6,江西省,江西,771,0,102,1,,360000,"[{'cityName': '南昌', 'confirmedCount': 189, 'su..."
7,江苏省,江苏,492,0,72,0,,320000,"[{'cityName': '南京', 'confirmedCount': 78, 'sus..."
8,重庆市,重庆,468,0,51,2,,500000,"[{'cityName': '万州区', 'confirmedCount': 97, 'su..."
9,山东省,山东,459,0,63,1,,370000,"[{'cityName': '青岛', 'confirmedCount': 52, 'sus..."


In [8]:
provinces_df = dict()
for idx in country_df.index:
    se = country_df.iloc[idx]
    if se['cities']:
        provinces_df[se['provinceName']] = pd.read_json(json.dumps(se['cities']))
pprint(provinces_df)

 10               0           0          0   
2       兵团第九师               4               0           0          0   
3         昌吉州               3               0           0          0   
4          巴州               3               0           0          0   
5       兵团第四师               3               0           0          0   
6         吐鲁番               1               0           0          0   
7       兵团第七师               1               0           0          0   
8   兵团第八师石河子市               1               0           0          0   
9       阿克苏地区               1               0           0          0   
10  兵团第六师五家渠市               1               0           0          0   
11     兵团第十二师               1               0           0          0   

    locationId  
0       650100  
1       654000  
2            0  
3            0  
4       511902  
5            0  
6       650400  
7            0  
8            0  
9       652900  
10           0  
11           0  ,
 '江苏省':    

In [16]:
se = country_df['provinceName']
dir(se)

['T',
 '_AXIS_ALIASES',
 '_AXIS_IALIASES',
 '_AXIS_LEN',
 '_AXIS_NAMES',
 '_AXIS_NUMBERS',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__

In [24]:
len(country_df)

34

In [30]:
name = 3
f'{name}'

'3'

In [32]:
country_df.iloc[3]

provinceName                                                       河南省
provinceShortName                                                   河南
confirmedCount                                                    1073
suspectedCount                                                       0
curedCount                                                         169
deadCount                                                            6
comment                                           参考卫健委统计口径，部分县区与地级市合并
locationId                                                      410000
cities               [{'cityName': '信阳', 'confirmedCount': 220, 'su...
Name: 3, dtype: object

In [47]:
FileNotFoundError('aa')

FileNotFoundError('aa')